# **Import libraries**

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow 
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications import VGG19 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers,Sequential,regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, AveragePooling2D, Dropout,BatchNormalization 
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.optimizers import Adam

# **Preparing dataset**

In [ ]:
train_path="../input/Kannada-MNIST/train.csv"
test_path="../input/Kannada-MNIST/test.csv"

In [ ]:
train_df=pd.read_csv(train_path)
train_df.head()

In [ ]:
train_df.label.value_counts()

This is a balanced dataset. All the numbers have 6000 images each

In [ ]:
image= train_df.iloc[1,1:].values.reshape(28,28)
plt.imshow(image,cmap='gray')

In [ ]:
def create_k_folds(kfolds,train_df):
    train_df.loc[:,'kfold']=-1
    train_df.sample(frac=1).reset_index(drop=True)
    sf=StratifiedKFold(n_splits=kfolds)
    y=train_df.label.values
    for fold,(t_,v_) in enumerate(sf.split(train_df,y)):
        train_df.loc[v_,'kfold']=fold
    return train_df

In [ ]:
def create_train_valid_set(fold,df):
    train_set=df[df.kfold!=fold].reset_index(drop=True)
    valid_set=df[df.kfold==fold].reset_index(drop=True)
    X_train=train_set.drop(['label','kfold'],axis=1)
    y_train=train_set.label
    X_valid=valid_set.drop(['label','kfold'],axis=1)
    y_valid=valid_set.label
    X_train=X_train.values.reshape(-1,28,28,1)
    X_valid=X_valid.values.reshape(-1,28,28,1)
    print('The shape of train set now is',X_train.shape)
    print('The shape of valid set now is',X_valid.shape)
    y_train=to_categorical(y_train,num_classes=10)
    y_valid=to_categorical(y_valid,num_classes=10)
    return X_train,y_train,X_valid,y_valid
    
    

In [ ]:
def create_train_val_generator(X_train,X_valid,y_train,y_valid):
    train_gen= ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  zoom_range=0.15,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.15,
                                  horizontal_flip=True,
                                  fill_mode="nearest").flow(X_train,y_train,batch_size=64)
    valid_gen= ImageDataGenerator(rescale=1./255).flow(X_valid,y_valid,batch_size=64)
    return train_gen,valid_gen
    

# **Making Model**

In [ ]:
def make_model():
    model=Sequential()
    model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (28,28,1)))
    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(BatchNormalization(momentum=.15))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))


    model.add(Conv2D(filters = 256, kernel_size = (3,3), activation ='relu'))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), activation ='relu'))
    model.add(BatchNormalization(momentum=0.15))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.5))

    model.add(AveragePooling2D())
    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.4))
    model.add(Dense(10, activation = "softmax"))
    
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy',tensorflow.keras.metrics.AUC(name='auc')])
    
    return model

In [ ]:
Epochs=100
Batch_size=64
my_callbacks = [EarlyStopping(patience=3,monitor='val_loss', mode='min',restore_best_weights=True,verbose=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=2, min_lr=0.00001, mode='auto',verbose=1)]

# **Training the model**

In [ ]:
def run_train(kfold):
    train=create_k_folds(kfold,train_df)
    for i in range(kfold):
        X_train,y_train,X_valid,y_valid=create_train_valid_set(i,train)
        train_gen,val_gen=create_train_val_generator(X_train,X_valid,y_train,y_valid)
        model=make_model()
        print('#'*25)
        print(f'{i}th fold training')
        print('#'*25)
        history=model.fit_generator(train_gen,
                              epochs =20, validation_data=val_gen,
                              verbose = 1, steps_per_epoch=X_train.shape[0] // Batch_size
                              ,callbacks=my_callbacks)
        model.save(f'model_{i}.h5')

In [ ]:
run_train(5)

**Model with k fold = 3 works the best**

In [ ]:
model=tensorflow.keras.models.load_model('model_3.h5')

In [ ]:
test_df=pd.read_csv(test_path)

In [ ]:
test_df.head()

In [ ]:
test=test_df.drop('id',axis=1)
test=test/255
test=test.values.reshape(-1,28,28,1)

In [ ]:
y_pred=model.predict(test)     
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
sub=pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
sub['label']=y_pred
sub.to_csv('submission.csv',index=False)

In [ ]:
sub.head(10)